In [ ]:
from pathlib import Path

from linopy import Model
import pandas as pd

# Load data

In [ ]:
MAX_TIMESTEPS = 500
assert MAX_TIMESTEPS % 2 == 0

In [ ]:
ROOT_DIR = Path.cwd()
DATA_DIR = ROOT_DIR / "data"

battery_parameters = pd.read_csv(
    DATA_DIR / "battery_parameters.csv",
    index_col=0,
)["Values"].to_dict()
half_hourly_market_series = pd.read_csv(
    DATA_DIR / "half-hourly-market.csv",
    index_col=0,
    skiprows=1,
    nrows=MAX_TIMESTEPS,
    names=["Price (£/MWh)"],
).iloc[:, 0]
half_hourly_market_series.index = pd.to_datetime(half_hourly_market_series.index, format="%d/%m/%Y %H:%M")
hourly_market_series = pd.read_csv(
    DATA_DIR / "hourly-market.csv",
    index_col=0,
    skiprows=1,
    nrows=MAX_TIMESTEPS/2,
    names=["Price (£/MWh)"],
).iloc[:, 0]
hourly_market_series.index = pd.to_datetime(hourly_market_series.index, format="%d/%m/%Y %H:%M")

aligned_hourly_market_series = pd.concat([
    hourly_market_series,
    hourly_market_series.set_axis(hourly_market_series.index + pd.to_timedelta(30, unit='m')),
]).sort_index()

# Form model - simple case, participating in one market

In [ ]:
m = Model(
    force_dim_names=True  # add this in later for safety
)

Index:

In [ ]:
time = pd.Index(half_hourly_market_series.index, name="time")

Coefficients:
$$
\begin{array}{lll}
    \text{price at time $t$} & p_t & \\
    \text{discharge efficiency} & e^d & = 0.95 \\
    \text{charge efficiency} & e^c & = 0.95 \\
\end{array}
$$

Variables:
$$
\begin{array}{lll}
    \text{discharging active} & s^d_t & \in \set{0, 1} \\
    \text{charging active} & s^c_t & \in \set{0, 1} \\
    \text{discharge rate at time $t$} & r^d_t \\
    \text{charge rate at time $t$} & r^c_t \\
\end{array}
$$

where:
$$
\begin{array}{ll}
    0 \leq r^d_t \leq r^d_{max} & \text{(up to maximum discharge rate)} \\
    0 \leq r^c_t \leq r^c_{max} & \text{(up to maximum charge rate)} \\
\end{array}
$$

In [ ]:
price_30min = half_hourly_market_series

charge_efficiency = 1 - battery_parameters["Battery charging efficiency"]
discharge_efficiency = 1 - battery_parameters["Battery discharging efficiency"]

is_charging = m.add_variables(
    binary=True,
    coords=[time],
    name="is charging",
)
is_discharging = m.add_variables(
    binary=True,
    coords=[time],
    name="is discharging",
)

charge_rate = m.add_variables(
    lower=0,
    upper=battery_parameters["Max charging rate"],
    coords=[time],
    name="charge rate",
)

discharge_rate = m.add_variables(
    lower=0,
    upper=battery_parameters["Max discharging rate"],
    coords=[time],
    name="discharge rate",
)

In [ ]:
timestep_duration = 0.5
initial_stored_energy = 0.0
max_stored_energy = battery_parameters["Max storage volume"]
stored_energy = initial_stored_energy + timestep_duration * (charge_rate * charge_efficiency - discharge_rate).shift(time=1).cumsum()

In [ ]:
m.add_constraints(is_charging + is_discharging <= 1, name="charging and discharging mutually exclusive");

In [ ]:
m.add_constraints(charge_rate <= is_charging * battery_parameters["Max charging rate"], name="cannot exceed max charge rate");

In [ ]:
m.add_constraints(discharge_rate <= is_discharging * battery_parameters["Max discharging rate"], name="cannot exceed max discharge rate");

In [ ]:
m.add_constraints(timestep_duration * discharge_rate <= stored_energy, name="cannot discharge more than stored energy");

In [ ]:
m.add_constraints(timestep_duration * charge_rate <= max_stored_energy - stored_energy, name="cannot charge more than spare capacity");

We wish to optimise profit:
$$
\max \sum_t^n p_t (s^d_t r^d_t e^d - s^c_t r^c_t / e^c)
$$

In [ ]:
m.add_objective(
    price_30min * (discharge_rate / discharge_efficiency - charge_rate * charge_efficiency),
    sense="max",
)

subject to:
$$
\begin{array}{ll}
s^d_t + s^c_t \leq & \text{charging and discharging cannot occur simultaneously} \\
\delta r^d_t \leq E_t & \text{cannot discharge more than the stored energy in a given timestep} \\
\delta r^c_t \leq E_{max} - E_t & \text{cannot charge more than the remaining capacity in a given timestep} \\
E_t = E_{init} + \delta \sum_{i=0}^{i=t-1} (r^c_i/e^c - r^d_i) & \text{stored energy is the cumulative sum of all prior charging and discharging events} \\
0 \leq E_t \leq E_t^{max} & \text{(up to maximum storage capacity)}
\end{array}
$$

where:
$$
\begin{array}{ll}
    \text{duration of timestep} & \delta \\
    \text{stored energy at time $t$} & E_t \\
    \text{maximum storage volume} & E_{max} \\
    \text{initial stored energy} & E_{init} \\
\end{array}
$$

In [ ]:
m.solve(solver_name="highs")

In [ ]:
m.solution

# Validation

In [ ]:
assert ((charge_rate.solution * discharge_rate.solution).round(8) == 0).all()

In [ ]:
oops = charge_rate.solution * discharge_rate.solution != 0

In [ ]:
df = pd.DataFrame({
    "price": price_30min,
    "charge rate": charge_rate.solution.values,
    "discharge rate": discharge_rate.solution.values,
    "stored energy": stored_energy.solution.values,
    "energy added": charge_rate.solution.values * charge_efficiency * timestep_duration,
    "energy removed": discharge_rate.solution.values * timestep_duration,
    "energy sold": discharge_rate.solution.values * timestep_duration * discharge_efficiency,
}, index=time)

df.head(5)

# Visualisation

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
fig, (axis_price, axis_energy, axis_flow) = plt.subplots(3, figsize=(12, 6), sharex=True)
axis_energy.set_ylim(0, 1.1*battery_parameters["Max storage volume"])
axis_flow.set_ylim(-1.25*battery_parameters["Max discharging rate"], 1.25*battery_parameters["Max charging rate"])
axis_price.set_ylabel("£")
axis_energy.set_ylabel("E")
axis_flow.set_ylabel("r")
axis_price.plot(half_hourly_market_series, color="black")
axis_energy.fill_between(
    time,
    stored_energy.solution.values,
    color="blue",
)
axis_flow.fill_between(
    time,
    m.solution.variables["charge rate"],
    step="post",
    color="green",
    label="charging",
)
axis_flow.fill_between(
    time,
    -m.solution.variables["discharge rate"],
    step="post",
    color="red",
    label="discharging",
)
axis_flow.legend()